Diagonalizing the Hamiltonian matrix for the Rydberg hamiltonian to find the energy eigenvalues and eigenkets. Calculate the groundstate magnetization and energy at zero temperature and finite temperature.

$$
H = -\sum_{\langle i j \rangle} V_{ij} (\sigma^z_i \sigma^z_j + \sigma^z_i + \sigma^z_j) - \Omega \sum_{i=1}^N \sigma^x_i - \frac{h}{2} \sum_{i=1}^N \sigma^z_i - \left(\frac{Nh}{2} + \sum_{\langle i j \rangle} V_{ij}\right)
$$
where ${\bf \sigma}_i$ are Pauli operators and $V_{ij} = \frac{C}{R_{ij}^6} = \frac{1}{r^6}\frac{C}{|i-j|^6}$ where $r$ is the nearest neighbour length.

In [1]:
using LinearAlgebra

N = 2
Dim = 2^N

r = 1. # this is the nearest neighbour length scale
C = 1.
Ω = 0.5 # this is the transverse field
h = 0.5 # this is the longitudinal field

Hamiltonian = zeros(Dim,Dim)   #This is your 2D Hamiltonian matrix

function Vij(i,j)
    return C / ((r*abs(i-j))^6)
end

# Calculate the sum of all Vij's for the diagonal shift
sumVij = 0
for i = 1:N-1
    for j = (i+1):(N)
        sumVij += Vij(i,j)
    end
end    

for Ket = 0:Dim-1  #Loop over Hilbert Space
    Diagonal = 0.
    for SpinIndex1 = 0:N-2  #Loop over spin index (base zero, stop one spin before the end of the chain)
        Spin1 = 2*((Ket>>SpinIndex1)&1) - 1
        
        for SpinIndex2 = (SpinIndex1+1):(N-1)
            Spin2 = 2*((Ket>>SpinIndex2)&1) - 1
            # Vij term
            Diagonal = Diagonal - Vij(SpinIndex1,SpinIndex2)*(Spin1*Spin2 + Spin1 + Spin2) #spins are +1 and -1
        end
        
        # Longitudinal Field term
        Diagonal = Diagonal - 0.5*h*Spin1
    end
    SpinN = 2*((Ket>>(N-1))&1) - 1
    Diagonal = Diagonal - 0.5*h*SpinN
    # add constant shift terms
    Diagonal = Diagonal - 0.5*N*h - sumVij
    
    Hamiltonian[Ket+1,Ket+1] = Diagonal
    
    # transverse field term
    for SpinIndex = 0:N-1
        bit = 2^SpinIndex   #The "label" of the bit to be flipped
        Bra = Ket ⊻ bit    #Binary XOR flips the bit
        Hamiltonian[Bra+1,Ket+1] = -Ω
    end 
end

In [2]:
### ZERO-T GROUND STATE ###

Diag = eigen(Hamiltonian)     #Diagonalize the Hamiltonian
GroundState = Diag.vectors[:, 1];  #this gives the groundstate eigenvector
E0 = Diag.values[1] / N
println("E0 / N = ",E0)

magnetization = 0
for Ket = 0:Dim-1  #Loop over Hilbert Space
    SumSz = 0.
    for SpinIndex = 0:N-1  #Loop over spin index (base zero, stop one spin before the end of the chain)
        Spin1 = 2*((Ket>>SpinIndex)&1) - 1
        SumSz += Spin1 #spin is +1 or -1
    end
    magnetization += SumSz*SumSz*GroundState[Ket+1]^2  #Don't forgot to square the coefficients...
end

println("Mag. / N^2 = ", magnetization/(N*N))

E0 / N = -2.5553960170015677
Mag. / N^2 = 0.9760494848129526


In [3]:
### FINITE-T ###

β = 1
H = zeros(Dim, Dim)
for i = 1:Dim
    H[i,i] = Diag.values[i]
end
    
expH = exp.(-β*H)
Z = tr(expH) # partition function
ρ = expH / Z

sigmaZ1 = [1 0; 0 -1]
sigmaZN = kron(sigmaZ1, sigmaZ1)
for i = 3:N
    sigmaZN = kron(sigmaZN, sigmaZ1)
end

m_thermal = tr(sigmaZN*ρ) / (N*N)
E_thermal = tr(H*ρ) / N

println("E thermal / N = ", E_thermal)
println("Mag. / N^2 = ", m_thermal)

E thermal / N = -2.4925433961464405
Mag. / N^2 = 0.23780569199821283
